In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/home/torsten/cpp/build/aru_sil_core/bin/output.txt", header=None)
df

,0,1,2,3,4
0,1,0,-7.349040e-17,-2.862980e-18,9.093930e-17
1,1,1,7.924640e-01,-1.069570e-02,-7.359030e+00
2,2,0,-9.033160e-01,-3.006680e-02,-1.416610e+00
3,2,1,-4.556590e-01,-2.248410e-02,1.092180e-01
4,2,2,-4.419920e+00,-5.701960e-01,-1.515010e+00
...,...,...,...,...,...
855,40,36,2.105660e+01,2.120900e+00,-2.177800e+00
856,40,37,2.068530e+01,1.712120e+00,-2.223610e+00
857,40,38,1.889750e+01,6.112860e-01,-2.362600e+00
858,40,39,1.707900e+01,-4.878000e-01,-2.515170e+00


In [3]:
%matplotlib tk
start = 0

plt.figure()
plt.axis('equal')
for i in range(45):
    plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x")
    plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

plt.show()

IndexError: index 901 is out of bounds for axis 0 with size 860